## *Expert System based on predicate logic using **pyDatalog** library.*
*The system will infer family realationships based on a family tree in a csv file*

***Steps:***

1- Install pyDatalog library using pip.

2- Import pyDatalog and csv library.

3- Create terms using pyDatalof.create_terms().

4- Open csv file in read mode then load its contents into variable **rows** usin csv.DictReader class.

5- Loop throw **rows** and load facts to agent's ***Knowledge Base*** .

6- Building rules that agent will use for reasoning.

7- Initiate queries to assert facts.

## pyDatalog Module Installation

In [ ]:
# Install pyDatalog
%pip install pyDatalog

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.6 MB/s eta 0:00:00


## Modules Importing

In [ ]:
# Import pyDatalog & csv libraries
from csv import DictReader
from pyDatalog import pyDatalog as dl

## Terms Defintion

In [ ]:
# Defien terms
# s_father_of: Step father
# f_inlaw_of: Father in law
# m_inlaw_of: Mother in law
# b_inlaw_of: Brother in law
# s_inlaw_of: Sister in law
# n_inlaw_of: Nephew in law
# nc_inlaw_of: Niece in law
# g_father_of: Grand father
# g_mother_of: Grand mother
dl.create_terms(
    '''
    parent_of, maried_to, gender_of, father_of,
    mother_of, brother_of, sister_of, s_father_of,
    cousin_of,aunt_of, uncle_of, nephew_of, niece_of,
    f_inlaw_of, m_inlaw_of, b_inlaw_of,s_inlaw_of,
    n_inlaw_of, nc_inlaw_of, g_father_of, g_mother_of,
    ancestor_of, X, Y, Z, W
    '''
)

## Loading Facts

In [ ]:
# Open csv file read the contents and load them as facts to Agent's KB
with open('data.csv') as csvfile:
  rows = DictReader(
      csvfile,
      fieldnames=['person', 'father', 'mother', 'gender'],
      delimiter=";"
      )

  for row in rows:
    # Facts
    +gender_of(row['person'], row['gender'])
    +father_of(row['person'], row['father'])
    +mother_of(row['person'], row['mother'])
    +maried_to(row['father'], row['mother'])


## Rules Definition

In [ ]:
# Parents rules
parent_of(X, Y) <= father_of(X, Y)
parent_of(X, Y) <= mother_of(X, Y)
###############################################################################
# Brothers & Sisters rules
# Brother: Same parents and is male
brother_of(X, Y) <= parent_of(X, Z) & parent_of(Y, Z) & ~(X==Y) & gender_of(Y, 'male')
# Sister:  Same parents and is female
sister_of(X, Y) <= parent_of(X, Z) & parent_of(Y, Z) & ~(X==Y) & gender_of(Y, 'female')
###############################################################################
# Step father rule
# Step father: maried to person's mother bunt not hes\her father
s_father_of(X, Y) <= mother_of(X, Z) & maried_to(Y, Z) & ~(father_of(X, Y))
##############################################################################
# Cousin: fathers are brothers
cousin_of(X, Y) <= father_of(X, Z) & father_of(Y, W) & brother_of(Z, W)
##############################################################################
# Aunt & Uncle rules
# Aunt: siter of father or mother
aunt_of(X, Y) <= parent_of(X, Z) & sister_of(Z, Y)
# Uncle: brother of father or sister
uncle_of(X, Y) <= parent_of(X, Z) & brother_of(Z, Y)
##############################################################################
# Nephew & Niecw rules
# Nephew: child of sister or child of brother and the child is male
nephew_of(X, Y) <= uncle_of(Y, X) & gender_of(Y, 'male')
nephew_of(X, Y) <= aunt_of(Y, X)  & gender_of(Y, 'male')
# Niece: child of sister or child of brother and the child is female
niece_of(X, Y) <= uncle_of(Y, X) & gender_of(Y, 'female')
niece_of(X, Y) <= aunt_of(Y, X)  & gender_of(Y, 'female')
##############################################################################
# In-law rules
# Father in law: father of the husbend or the wife (x,y or y,x)
f_inlaw_of(X, Y) <= maried_to(X, Z) & father_of(Z, Y)
f_inlaw_of(X, Y) <= maried_to(Z, X) & father_of(Z, Y)
# Mother in law: mother of the husbend or the wife (x,y or y,x)
m_inlaw_of(X, Y) <= maried_to(X, Z) & mother_of(Z, Y)
m_inlaw_of(X, Y) <= maried_to(Z, X) & mother_of(Z, Y)

# Brother in law: sister's husbend
b_inlaw_of(X, Y) <= sister_of(X, Z) & maried_to(Y, Z)
# Sister in law : brothe's wife
s_inlaw_of(X, Y) <= brother_of(X, Z) & maried_to(Z, Y)

# Nephew in law: male children of wife's brothers and sisters
n_inlaw_of(X, Y) <= maried_to(X, Z) & aunt_of(Y, Z) & gender_of(Y, 'male')
# Nephew in law: male children of husband's brothers and sisters
n_inlaw_of(X, Y) <= maried_to(Z, X) & uncle_of(Y, Z) & gender_of(Y, 'male')
# Niece in law: female children of wife's brothers and sisters
nc_inlaw_of(X, Y) <= maried_to(X, Z) & aunt_of(Y, Z) & gender_of(Y, 'female')
# Niece in law: female children of husband's brothers and sisters
nc_inlaw_of(X, Y) <= maried_to(Z, X) & uncle_of(Y, Z) & gender_of(Y, 'female')
#############################################################################
# Grandparents rules
# Grandfather: parent of the child parent and is male
g_father_of(X, Y) <= parent_of(X, Z) & parent_of(Z, Y) & gender_of(Y, 'male')
# Grandmather: parent of the child parent and is female
g_mother_of(X, Y) <= parent_of(X, Z) & parent_of(Z, Y) & gender_of(Y, 'female')
##############################################################################
# Ancestors rules
# Ancestor: parent of the person or parent of the person's parent
ancestor_of(X, Y) <= parent_of(X, Y)
ancestor_of(X, Y) <= parent_of(X, Z) & ancestor_of(Z, Y)

## Quiries

#### *Parents*

In [ ]:
print("Parent of Mohamad A | ", parent_of('Mohamad A', X))
print("\nFathr of Raheel | ", father_of('Raheel', X))
print("\nMother of Inana | ", mother_of('Inana', X))
print("\nKamal's children |", parent_of(X, 'Kamal'))

### *Brothers & Sisters*

In [ ]:
print("Brothers of Mohamad K | ", brother_of('Mohamad K', X))
print("\nSisters of Orwa | ", sister_of('Orwa', X))
print("\nBrother of Alia | ", brother_of('Alia', X))
print("\nSister's of Zelal |",sister_of('Zelal', X))

### *Step Father*

In [ ]:
print("Father of Samer | ", father_of('Samer', X))
print("\nStepfather of Samer |",s_father_of('Samer', X))
print("\nSaid is stepfather of |",s_father_of(X, 'Said'))

### *Cousines*

In [ ]:
print("Cousins of Ahmad K |",cousin_of('Ahmad K', X))
print("\nAli's Cousines | ", cousin_of(X, 'Ali'))

### *Uncles & Aunts*

In [ ]:
print("Uncle of Ali | ", uncle_of('Ali', X))
print("\nAunt of Lobana | ", aunt_of('Lobana', X))
print("\nYasser uncle of | ",uncle_of(X, 'Yasser'))
print("\nEnad aunt of | ", aunt_of(X, 'Enad'))

### *Nephew & Niece*

In [ ]:
print("Nephew of Ayman | ", nephew_of('Ayman', X))
print("\nNephew of Enad | ", nephew_of('Enad', X))
print("\nNiece of Yasser | ", niece_of('Yasser', X))
print("\nNiece of Zelal | ", niece_of('Zelal', X))

### *In Law*

In [ ]:
print("Father in law of Khadija H | ", f_inlaw_of('Khadija H', X))
print("\nMother in law of Najla | ", m_inlaw_of('Najla', X))
print("\nAhmad S is father in law of | ", f_inlaw_of(X, 'Ahmad S'))
print("\nBrother in law of Ayman |", b_inlaw_of('Ayman', X))
print("\nSister in law of Yasser |", s_inlaw_of('Yasser', X))

In [ ]:
print('Nephew in law of Mona | ', n_inlaw_of('Mona', X))
print('\nNiece in law of Jamal | ', nc_inlaw_of('Jamal', X))

### *Grandfather & Granmother*

In [ ]:
print("Grandfather of Zainab | ", g_father_of('Zainab', X))
print("\nMohamad is grandfather of | ", g_father_of(X, 'Mohamad'))
print("\nGrandmother of Lama | ", g_mother_of('Lama', X))
print("\nGhada is grandmother of | ", g_mother_of(X, 'Ghada'))

### *Ancestor*

In [ ]:
print("Ancestors of Inana | ", ancestor_of('Inana', X))
print("\nNoma is ancestor of | ", ancestor_of(X, 'Noma'))